In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [67]:
data = pd.read_csv("../../data/processed/data.csv",
                   dtype = {
                            "expensa_id": "object",
                            "unidad_id": "object",
                            "propietario_id": "object",
                            "inquilino_id": "object",
                            "consorcio_id": "object",
                            "expensa_padre_id": "object",
                            "unidad_prop": "object",
                            "consorcio_id": "object",
                            "usuario_creador_id": "object",
                            "concepto_id": "object",
                            "mes": "object",
                            "anio": "object",
                            "unidad_denominacion":"object",
                            "consorcio_nombre": "object",
                            "version":"object",
                            "concepto_id": "object",
                            "unidad_tipo": "object",
                            "modo_pago": "object",
                            "forma_prorrateo": "object",
                            "tipo_prorrateo": "object",
                            "metodo_pago": "object",                       
                    },
                    parse_dates = ["fecha_vencimiento_1", "fecha_vencimiento_2"])
data.fecha_pago = pd.to_datetime(data.fecha_pago, errors="coerce")

In [68]:
data["fecha_expensa"] = data.anio + '-' + data.mes + '-01'
data.fecha_expensa = pd.to_datetime(data.fecha_expensa)
data["fecha_expensa_anterior"] = data.fecha_expensa - pd.DateOffset(months=1)

In [69]:
data = data[['expensa_id', 'unidad_id', 'propietario_id', 'inquilino_id',
       'fecha_vencimiento_1', 'fecha_vencimiento_2', 'monto', 'monto_total',
       'monto_parcial', 'expensa_padre_id', 'int_dia', 'int_mes', 'mes',
       'anio', 'unidad_tipo',
       'modo_pago', 'forma_prorrateo', 'tipo_prorrateo', 'fecha_pago',
       'monto_pago', 'metodo_pago', 'fecha_expensa', 'fecha_expensa_anterior']]

In [70]:
data["impago"] = data.fecha_pago.isna()
data["pago_tardio_1"] = (data.fecha_pago > data.fecha_vencimiento_1) & (data.fecha_pago <= data.fecha_vencimiento_2)
data["pago_tardio_2"] = (data.fecha_pago > data.fecha_vencimiento_2)
data["pago_tardio"] = data.pago_tardio_1 | data.pago_tardio_2

data.loc[data.metodo_pago.isna(), "metodo_pago"] = "Impago"

data["propietario"] = data.propietario_id == "0"
data["inquilino"] = data.inquilino_id == "0"
data["venc_1_dayofweek"] = data.fecha_vencimiento_1.dt.dayofweek.map(str)
data["venc_1_day"] = data.fecha_vencimiento_1.dt.day
data["venc_1_fer"] = data.fecha_vencimiento_1.dt.date.map(lambda x: str(x) in feriados)
data["venc_2_dayofweek"] = data.fecha_vencimiento_2.dt.dayofweek.map(str)
data["venc_2_day"] = data.fecha_vencimiento_2.dt.day
data["venc_2_fer"] = data.fecha_vencimiento_2.dt.date.map(lambda x: str(x) in feriados)
data["pag_day"] = data.fecha_pago.dt.day
data["pag_dayofweek"] = data.fecha_pago.dt.dayofweek.map(str)

In [71]:
data_mes_anterior = data[["fecha_expensa", "unidad_id", "venc_1_dayofweek", "venc_1_day", "venc_1_fer", "venc_2_dayofweek", "venc_2_day", "venc_2_fer", "metodo_pago", "pag_day", "pag_dayofweek", "impago", "pago_tardio"]]

In [72]:
data.shape

(264541, 37)

In [73]:
data = pd.merge(
    data,
    data_mes_anterior,
    left_on=["fecha_expensa_anterior","unidad_id"],
    right_on=["fecha_expensa", "unidad_id"],
    how = "left",
    suffixes = ("","_ant"))

In [76]:
data = data[~data.fecha_expensa_ant.isna()]

In [77]:
data.columns

Index(['expensa_id', 'unidad_id', 'propietario_id', 'inquilino_id',
       'fecha_vencimiento_1', 'fecha_vencimiento_2', 'monto', 'monto_total',
       'monto_parcial', 'expensa_padre_id', 'int_dia', 'int_mes', 'mes',
       'anio', 'unidad_tipo', 'modo_pago', 'forma_prorrateo', 'tipo_prorrateo',
       'fecha_pago', 'monto_pago', 'metodo_pago', 'fecha_expensa',
       'fecha_expensa_anterior', 'impago', 'pago_tardio_1', 'pago_tardio_2',
       'pago_tardio', 'propietario', 'inquilino', 'venc_1_dayofweek',
       'venc_1_day', 'venc_1_fer', 'venc_2_dayofweek', 'venc_2_day',
       'venc_2_fer', 'pag_day', 'pag_dayofweek', 'fecha_expensa_ant',
       'venc_1_dayofweek_ant', 'venc_1_day_ant', 'venc_1_fer_ant',
       'venc_2_dayofweek_ant', 'venc_2_day_ant', 'venc_2_fer_ant',
       'metodo_pago_ant', 'pag_day_ant', 'pag_dayofweek_ant', 'impago_ant',
       'pago_tardio_ant'],
      dtype='object')

In [144]:
X = data[['monto', 'monto_total',
       'monto_parcial', 'int_dia', 'int_mes', 'mes',
       'anio', 'unidad_tipo', 'modo_pago', 'forma_prorrateo', 'tipo_prorrateo',
       'propietario', 'inquilino', 'venc_1_dayofweek',
       'venc_1_day', 'venc_1_fer', 'venc_2_dayofweek', 'venc_2_day',
       'venc_2_fer', 'venc_1_dayofweek_ant', 'venc_1_day_ant', 'venc_1_fer_ant',
       'venc_2_dayofweek_ant', 'venc_2_day_ant', 'venc_2_fer_ant',
       'metodo_pago_ant', 'pag_dayofweek_ant', 'impago_ant', 'pago_tardio_ant']]

In [145]:
X = pd.get_dummies(X)

In [146]:
y = data["pago_tardio"]

In [147]:
X_train = X.iloc[0:200000,:]
X_test = X.iloc[200001:,:]
y_train = y[0:200000]
y_test = y[200001:]

In [148]:
X_train.shape

(200000, 102)

In [149]:
X_test.shape

(46624, 102)

In [150]:
y_train.shape

(200000,)

In [151]:
X.isna().sum(axis=0)

monto                                   0
monto_total                             0
monto_parcial                           0
int_dia                                 0
int_mes                                 0
propietario                             0
inquilino                               0
venc_1_day                              0
venc_1_fer                              0
venc_2_day                              0
venc_2_fer                              0
venc_1_day_ant                          0
venc_2_day_ant                          0
mes_1                                   0
mes_10                                  0
mes_11                                  0
mes_12                                  0
mes_2                                   0
mes_3                                   0
mes_4                                   0
mes_5                                   0
mes_6                                   0
mes_7                                   0
mes_8                             

In [152]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 200,
                             max_depth = 20,
                             n_jobs = -1)

In [153]:
clf.fit(X_train, y_train )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [154]:
clf.score(X_train, y_train)

0.87181

In [155]:
clf.score(X_test, y_test)

0.8104838709677419

In [156]:
from sklearn.metrics import f1_score
f1_score(y_test, clf.predict(X_test))

0.6349363741530325

In [157]:
features_imp = pd.DataFrame( {"variable": X_train.columns,
               "importance": clf.feature_importances_})

In [158]:
features_imp.sort_values("importance", ascending= False)

,variable,importance
101,pago_tardio_ant_True,0.199070
100,pago_tardio_ant_False,0.172038
0,monto,0.072711
81,metodo_pago_ant_Efectivo,0.062691
1,monto_total,0.039137
4,int_mes,0.025516
9,venc_2_day,0.017817
7,venc_1_day,0.015667
3,int_dia,0.015132
12,venc_2_day_ant,0.014936


In [32]:
feriados = [
    #2013
    '2013-01-01',
    '2013-01-31',
    '2013-02-11',
    '2013-02-12',
    '2013-02-20',
    '2013-03-28',
    '2013-03-29',
    '2013-04-01',
    '2013-04-02',
    '2013-05-01',
    '2013-06-20',
    '2013-06-21',
    '2013-07-09',
    '2013-08-19',
    '2013-10-14',
    '2013-11-25',
    '2013-12-25',
    #2014
    '2014-01-01',
    '2014-03-03',
    '2014-03-04',
    '2014-03-24',
    '2014-04-02',
    '2014-04-18',
    '2014-05-01',    
    '2014-05-02',
    '2014-06-20',
    '2014-07-09',
    '2014-08-18',
    '2014-10-13',
    '2014-11-24',
    '2014-12-25',    
    '2014-12-26',
    #2015
    '2015-01-01',
    '2015-02-16',
    '2015-02-17',
    '2015-03-23',
    '2015-03-24',
    '2015-04-02',
    '2015-04-03',
    '2015-05-01',    
    '2015-05-25',
    '2015-06-20',
    '2015-07-09',
    '2015-08-17',
    '2015-10-12',
    '2015-11-23',    
    '2015-12-07',   
    '2015-12-08',
    '2015-12-25',
    #2016
    '2016-01-01',
    '2016-02-08',
    '2016-02-09',
    '2016-03-24',
    '2016-03-25',
    '2016-04-02',
    '2016-05-01',
    '2016-05-25',
    '2016-06-20',
    '2016-07-08',
    '2016-07-09',
    '2016-08-15',
    '2016-10-12',
    '2016-11-28',
    '2016-12-08',
    '2016-12-09',
    '2016-12-25',
    #2017
    '2017-01-01',
    '2017-02-27',
    '2017-02-28',
    '2017-03-27',
    '2017-04-02',
    '2017-04-14',
    '2017-05-01',
    '2017-05-25',
    '2017-06-17',
    '2017-06-19',
    '2017-07-09',
    '2017-08-21',
    '2017-10-09',
    '2017-11-27',
    '2017-12-08',
    '2017-12-25',
    #2018
    '2018-01-01',
    '2018-02-12',
    '2018-02-13',
    '2018-03-24',
    '2018-03-30',
    '2018-04-02',
    '2018-05-01',
    '2018-05-25',
    '2018-06-20',
    '2018-07-09',
    '2018-08-20',
    '2018-10-15',
    '2018-11-19',
    '2018-12-08',
    '2018-12-25'    
]